In [ ]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Dropout,Layer,GRU, BatchNormalization,RNN,SimpleRNN
from tensorflow.keras.optimizers import Adam,SGD
import tensorflow.keras.backend as K
from tensorflow.keras import initializers
drive.mount('/content/drive')


In [ ]:
import pickle
with open('//content/drive/MyDrive/Colab Notebooks/lstm_train_X.pkl', 'rb') as f:
    new_train_X=pickle.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/lstm_train_y.pkl', 'rb') as f:
    new_train_y=pickle.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/lstm_val_X.pkl', 'rb') as f:
    new_val_X=pickle.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/lstm_val_y.pkl', 'rb') as f:
    new_val_y=pickle.load(f)


In [ ]:
train_X = np.array(new_train_X[:,:,:17])
val_X = np.array(new_val_X[:,:,:17])

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
class TemporalAttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(TemporalAttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight',
                                 shape=(input_shape[-1], 1),
                                 initializer=initializers.he_normal(),
                                 trainable=True)
        super(TemporalAttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W))
        alpha = K.softmax(e, axis=1)
        # 변경: 원래 시퀀스와 가중치를 곱하여 출력 형태를 유지
        context = x * alpha
        return context

    def compute_output_shape(self, input_shape):
        return input_shape


In [ ]:
# 모델 구성을 위한 입력 정의
input_layer = Input(shape=(24, 17))

# 양방향 LSTM 레이어
x = Bidirectional(LSTM(1000, return_sequences=True, activation='tanh'))(input_layer)
x = Bidirectional(LSTM(500, return_sequences=True, activation='tanh'))(x)
#x = Bidirectional(LSTM(500, return_sequences=True, activation='tanh'))(x)
x = Bidirectional(LSTM(100, activation='tanh'))(x)

# 완전 연결 레이어
x = Dense(2048, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# 출력 레이어
output_layer = Dense(16)(x)

# 모델 생성
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# 모델 구조 시각화
#plot_model(model, to_file='improved_model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
'''
input_layer = Input(shape=(24, 24))

x = SimpleRNN(1000, return_sequences=True, activation='tanh')(input_layer)

# 두 번째 단방향 RNN 레이어
# 두 번째 RNN 레이어에서는 return_sequences=True를 제거하여 최종 출력만 생성합니다.
x = SimpleRNN(500, activation='tanh')(x)
# 완전 연결 레이어
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# 출력 레이어
output_layer = Dense(16)(x)

# 모델 생성
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
'''

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=100)
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')

model_his = model.fit(
    train_X,
    new_train_y,
    callbacks=[early_stopping, checkpoint],
    validation_data=(val_X, new_val_y),
    epochs=300,
    batch_size=128,
    shuffle=False
)

In [ ]:
plt.figure(figsize=(24, 6))
plt.plot(model_his.history['loss'][3:], label='Training_loss')
plt.plot(model_his.history['val_loss'][3:], label='Validation_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('Loss.png')
plt.show()
#model.save('lstm_model.h5')

In [ ]:
from keras.models import load_model

model = load_model('best_model.h5', custom_objects={'TemporalAttentionLayer': TemporalAttentionLayer})
loss = model.evaluate(val_X, new_val_y)
loss

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)


In [ ]:
model = tf.keras.models.load_model('/content/lstm_model.h5', custom_objects={'TemporalAttentionLayer': TemporalAttentionLayer})
loss = model.evaluate(new_val_X,new_val_y)
loss

In [ ]:
model = tf.keras.models.load_model('/content/lstm_best_model.h5')
loss = model.evaluate(new_val_X,new_val_y)
loss